In [15]:
import pandas as pd
from summarytools import dfSummary
#from pandas.io.formats.style import Styler

In [4]:
data = pd.read_csv('../data/raw/HR_Attrition_Dataset.csv')
print(data.head())

   Age Attrition     BusinessTravel  DailyRate              Department  \
0   41       Yes      Travel_Rarely       1102                   Sales   
1   49        No  Travel_Frequently        279  Research & Development   
2   37       Yes      Travel_Rarely       1373  Research & Development   
3   33        No  Travel_Frequently       1392  Research & Development   
4   27        No      Travel_Rarely        591  Research & Development   

   DistanceFromHome  Education EducationField  EmployeeCount  EmployeeNumber  \
0                 1          2  Life Sciences              1               1   
1                 8          1  Life Sciences              1               2   
2                 2          2          Other              1               4   
3                 3          4  Life Sciences              1               5   
4                 2          1        Medical              1               7   

   ...  RelationshipSatisfaction StandardHours  StockOptionLevel  \
0  ...

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1470 non-null   int64 
 1   Attrition                 1470 non-null   object
 2   BusinessTravel            1470 non-null   object
 3   DailyRate                 1470 non-null   int64 
 4   Department                1470 non-null   object
 5   DistanceFromHome          1470 non-null   int64 
 6   Education                 1470 non-null   int64 
 7   EducationField            1470 non-null   object
 8   EmployeeCount             1470 non-null   int64 
 9   EmployeeNumber            1470 non-null   int64 
 10  EnvironmentSatisfaction   1470 non-null   int64 
 11  Gender                    1470 non-null   object
 12  HourlyRate                1470 non-null   int64 
 13  JobInvolvement            1470 non-null   int64 
 14  JobLevel                

In [10]:
pd.set_option('display.max_columns', None) 
data.describe() 


,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,MonthlyRate,NumCompaniesWorked,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
count,1470.000000,1470.000000,1470.000000,1470.000000,1470.0,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.0,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000
mean,36.923810,802.485714,9.192517,2.912925,1.0,1024.865306,2.721769,65.891156,2.729932,2.063946,2.728571,6502.931293,14313.103401,2.693197,15.209524,3.153741,2.712245,80.0,0.793878,11.279592,2.799320,2.761224,7.008163,4.229252,2.187755,4.123129
std,9.135373,403.509100,8.106864,1.024165,0.0,602.024335,1.093082,20.329428,0.711561,1.106940,1.102846,4707.956783,7117.786044,2.498009,3.659938,0.360824,1.081209,0.0,0.852077,7.780782,1.289271,0.706476,6.126525,3.623137,3.222430,3.568136
min,18.000000,102.000000,1.000000,1.000000,1.0,1.000000,1.000000,30.000000,1.000000,1.000000,1.000000,1009.000000,2094.000000,0.000000,11.000000,3.000000,1.000000,80.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,30.000000,465.000000,2.000000,2.000000,1.0,491.250000,2.000000,48.000000,2.000000,1.000000,2.000000,2911.000000,8047.000000,1.000000,12.000000,3.000000,2.000000,80.0,0.000000,6.000000,2.000000,2.000000,3.000000,2.000000,0.000000,2.000000
50%,36.000000,802.000000,7.000000,3.000000,1.0,1020.500000,3.000000,66.000000,3.000000,2.000000,3.000000,4919.000000,14235.500000,2.000000,14.000000,3.000000,3.000000,80.0,1.000000,10.000000,3.000000,3.000000,5.000000,3.000000,1.000000,3.000000
75%,43.000000,1157.000000,14.000000,4.000000,1.0,1555.750000,4.000000,83.750000,3.000000,3.000000,4.000000,8379.000000,20461.500000,4.000000,18.000000,3.000000,4.000000,80.0,1.000000,15.000000,3.000000,3.000000,9.000000,7.000000,3.000000,7.000000
max,60.000000,1499.000000,29.000000,5.000000,1.0,2068.000000,4.000000,100.000000,4.000000,5.000000,4.000000,19999.000000,26999.000000,9.000000,25.000000,4.000000,4.000000,80.0,3.000000,40.000000,6.000000,4.000000,40.000000,18.000000,15.000000,17.000000


In [4]:
data.describe(include='object')

,Attrition,BusinessTravel,Department,EducationField,Gender,JobRole,MaritalStatus,Over18,OverTime
count,1470,1470,1470,1470,1470,1470,1470,1470,1470
unique,2,3,3,6,2,9,3,1,2
top,No,Travel_Rarely,Research & Development,Life Sciences,Male,Sales Executive,Married,Y,No
freq,1233,1043,961,606,882,326,673,1470,1054


From this initial inspection what immediately stands out is that we have 3 constant features: "EmployeeCount", "StandardHours", and "Over18". We can remove those straight away. Additionally, the employee number (ID) feature, does not seem to contain any relevant info, and  we'll drop it too.

In [19]:
data.drop(columns=['EmployeeCount','Over18','StandardHours','EmployeeNumber'],inplace=True)

KeyError: "['EmployeeCount', 'Over18', 'StandardHours', 'EmployeeNumber'] not found in axis"

In [ ]:
cat_cols = data.select_dtypes(include=["object"]).columns

for col in cat_cols: 
    print(f"Value counts for column '{col}':")
    print(data[col].value_counts())
    print("\n") 

Value counts for column 'Attrition':
Attrition
No     1233
Yes     237
Name: count, dtype: int64


Value counts for column 'BusinessTravel':
BusinessTravel
Travel_Rarely        1043
Travel_Frequently     277
Non-Travel            150
Name: count, dtype: int64


Value counts for column 'Department':
Department
Research & Development    961
Sales                     446
Human Resources            63
Name: count, dtype: int64


Value counts for column 'EducationField':
EducationField
Life Sciences       606
Medical             464
Marketing           159
Technical Degree    132
Other                82
Human Resources      27
Name: count, dtype: int64


Value counts for column 'Gender':
Gender
Male      882
Female    588
Name: count, dtype: int64


Value counts for column 'JobRole':
JobRole
Sales Executive              326
Research Scientist           292
Laboratory Technician        259
Manufacturing Director       145
Healthcare Representative    131
Manager                      102
Sale

In [ ]:
dfSummary(data)

No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,Age[int64],Mean (sd) : 36.9 (9.1)min < med < max:18.0 < 36.0 < 60.0IQR (CV) : 13.0 (4.0),43 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOnRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjEwLjYsIGh0dHBzOi8vbWF0cGxvdGxpYi5vcmcvq6yFwwAAAAlwSFlzAAAPYQAAD2EBqD+naQAAAmpJREFUeJzt3UGOokAYhuGqgZ4wmmCMie68wCS9cOkh5rBzhFl4Am/gzsQYjXQTEOnQ68mMAjZ8P++zx2bxNgGKqvJlWTqg7751fQLAPUJnlPf+h3Pue4OfyMqyfG/xlNBAaDXS2Wz2K47jad3fOJ/PR+/9b2LtB5OhVlfSKtL1ev0+mUzSRw8+nU7RZrOZHg6H6opMqD1gNdRPVaTz+fytzrFFUcTOudh7X+dwbhtaZjrUupIkecnz/HW5XAZBEDx8Rea2oX2E+hdZloVRFI1Wq1W6WCyOjxzLbcNzEOo/jMfjurcO1RsHtIj3qJBAqJBAqJBAqJAQGh0GjW+328sTTgkdCS0Og+Z5HhVF8TNN0z/OuVov/JsoiqL6J2GwYAihNhkG3e120+12O7per4H7YgwWDC/U2sOgx+Oxs/eYDBYMNFRVDBa0i6d+SCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSGAqSs80nMFqdhYrofZI0xmslmexEmqPNJnBan0WK6HamsFqdhYrD1OQQKiQQKiQEPZ4UzJW5MPzQ226KVnXK/JhOFfURpuSdbkiHwb4eqrupmRdrsinrDC6NivvUQ1JDK/NSqiGZIbXZiVUg8YG12YlVEh8uUWokPhyi1Ah8eUWoULiyy3G+iHhv1dUds9D70NV3j0Pw7qiSu6eB3vDt6HF3fNgb/iWp35IDN8SKiSGb3k9BQmECgmECgmECgmECgmECgmECgmECgnhvSMHj/7w5XL5PCZJkmi/34++8njVv6163q7h8ff09QH5NMarfzIoawAAAABJRU5ErkJggg=="">",0(0.0%)
2,Attrition[object],1. No2. Yes,"1,233 (83.9%)237 (16.1%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAAuCAYAAAA/ZmtKAAAAOnRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjEwLjYsIGh0dHBzOi8vbWF0cGxvdGxpYi5vcmcvq6yFwwAAAAlwSFlzAAAPYQAAD2EBqD+naQAAASpJREFUeJzt2z1qw0AUReGZIEzSSBiBFqNFeLFehHYjglTEwc2EKbyEd3gO5yvcTuHjX+bW1lqRCLWU8lVKuSCnKbNna+0RecAwz/NtHMdr5CHK7zzP71rrPTK4oYe2rutjmqbfqEOU23Ecn9u2Xfd9759wcbH1hx7asiw/UYfoLfSvU6E+og+QXoxNGGMTxtiEMTZhjE0YYxPG2IQxNmGMTRhjE8bYhDE2YYxNGGMTZnhdnuOOVDbU8z/068D9liZxeU559Q76DiHyDAcvwgYv1SmfKNQ7W/irRvkhUz5iJqb8wqd81ExM+VFTPn/pyj91xTE2YYxNGGMTxtiEMTZhjE0YYxPG2IQxNmGMTRhjE8bYhDE2YYxN/2fK50xQ6JSPmIkpPwcvKpQ/Cltq3zMouvQAAAAASUVORK5CYII="">",0(0.0%)
3,BusinessTravel[object],1. Travel_Rarely2. Travel_Frequently3. Non-Travel,"1,043 (71.0%)277 (18.8%)150 (10.2%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAABFCAYAAABdVZTTAAAAOnRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjEwLjYsIGh0dHBzOi8vbWF0cGxvdGxpYi5vcmcvq6yFwwAAAAlwSFlzAAAPYQAAD2EBqD+naQAAAYNJREFUeJzt20FqwlAYhdGXIqWdKCK4GBfRxXYR2Y2IGWhxkvJ2oAg3/HrOoNME+tWY8u4wz3ODhI/IVaC1NrTWvltrn0vfCA+7zfN8bYWsdrvdz3q93i59IzxmmqbTMAy/lYJb9dAOh8N1s9n8LX0z3Od8Pn+N47g9Ho/9iVQntv6jh7bf7y9L3wwP6V9/SvGCQIzYiBEbMWIjRmzEiI0YsREjNmLERozYiBEbMWIjRmzEiI3sEaN+Pip3SZ5V9fe16ic++0G8iuej3tk0Tad+NLwVYoNQ163SkfBuMOXj1T/Zyv1VUnRdVXEZRMF1VdVlEHXXVd5835B/6hIjNmLERozYiBEbMWIjRmzEiI0YsREjNmLERozYiBEbMWLjdddVVZdBFF1XVVwG8TwbBGKsq4jxgkCpx6hHIpkpn1kekSmfWR7pKZ9ZHnfxgkCM2IgRGzFiI0ZsxIiNGLERIzZixEaM2IgRGzFiI0ZsxIiNGlM+szyiUz6zPO5lg0CMKR8t5R9KvKNKejbZqwAAAABJRU5ErkJggg=="">",0(0.0%)
4,DailyRate[int64],Mean (sd) : 802.5 (403.5)min < med < max:102.0 < 802.0 < 1499.0IQR (CV) : 692.0 (2.0),886 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOnRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjEwLjYsIGh0dHBzOi8vbWF0cGxvdGxpYi5vcmcvq6yFwwAAAAlwSFlzAAAPYQAAD2EBqD+naQAAApFJREFUeJzt3T+O2kAYh+GZACsDii3ECjousFIKSprcIIdNmy5IcAFuQIOQEBjhLMIYItOlCnj4Mz/7fXrDyHplzzdotfZ8PhvAd19evQDgGnVTUtbapjHmzeEjDufz+fOOS4KDelkj7Xa7P8Iw7BT9jO12u7bW/iRWP5Qy1PxJmkc6Go0+oyja33pxHMfBZDLprFar/IlMqB4oa6gXeaS9Xu9PwcvzrUPVtjwHX98gpQ61aqzjlsfn7Q6hlstb0S1Pvt0Zj8e91Wr1bq3d+vZEJtQSigpseZIkaaRp+m0wGNRqtdrN+/pHP5EJFReHw6EeBEFrOBzu+/3+2ng2gBIq/tFut70cQOtVnSIfiXv25FDLPEU+CvfsNU9Upymyoofmlb1nWZY1jDGhtfbub5L6Iw/OsywLHRb+0tegw00PT6dTI4qidcG9nuQPDYnjqcH/3iQPG6bucdyx2Wx21tpfxph9kVjMC9aepmmQZdnHfr//bYwpOpRU6tQgvuJNUvf1uGOxWHydTqffB4PB+7NjcVn7fD7vzGaz1vF4rJkKahc/NWi+9Hiq6MLX63Xz1bEUWXu+bpfvhOg5KrFAItSqcZycQ5e9uc8I1SOuA2ha4kGOUD3iOoDOSzzIEaqHXAZQU1L8FSokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECrK82/Q4zgO